## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

#### Best Practises
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW,TFIDF,Word2vec
4. Train ML algorithms

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the dataset
import pandas as pd
data=pd.read_csv('../Kindle Reviews/all_kindle_review.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
df=data[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [4]:
df.shape

(12000, 2)

In [5]:
## Missing Values
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [6]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [7]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [8]:
## Preprocessing And Cleaning

In [9]:
## postive review is 1 and negative review is 0
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

In [10]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [11]:
## 1. Lower All the cases
df['reviewText']=df['reviewText'].str.lower()

In [12]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [13]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\salma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [15]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub(r"[^a-zA-Z0-9\s]", "", x))

In [16]:
## Remove the stopwords
df['reviewText']=df['reviewText'].apply(
    lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')])
    )

In [17]:
## Remove url
df['reviewText']=df['reviewText'].apply(
    lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x))
    )

In [18]:
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

In [19]:
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

In [20]:
# lemmatize
df['reviewText']=df['reviewText'].apply(
    lambda x:" ".join([lemmatizer.lemmatize(word) for word in x.split()])
)

In [21]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [22]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'], test_size=0.20)

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    BaggingClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier
)

classifiers = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
    "SVM": LinearSVC(class_weight='balanced'),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced', random_state=42),
    "Extra Trees": ExtraTreesClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "Bagging": BaggingClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

scores = []
best_model = None
best_score = 0

for name, clf in classifiers.items():
    print(f"\n🧠 {name}")
    pipe = Pipeline([
        ('cv', CountVectorizer(max_features=3000, stop_words='english', ngram_range=(1, 2))),
        ('clf', clf)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    if acc > best_score:
        best_score = acc
        best_model = (name, pipe)

print("\n🏆 Best Model:")
print(f"{best_model[0]} with accuracy: {best_score:.4f}")


🧠 Naive Bayes
Accuracy: 0.8329166666666666
              precision    recall  f1-score   support

           0       0.74      0.79      0.76       825
           1       0.88      0.86      0.87      1575

    accuracy                           0.83      2400
   macro avg       0.81      0.82      0.82      2400
weighted avg       0.84      0.83      0.83      2400


🧠 Logistic Regression
Accuracy: 0.8195833333333333
              precision    recall  f1-score   support

           0       0.73      0.76      0.74       825
           1       0.87      0.85      0.86      1575

    accuracy                           0.82      2400
   macro avg       0.80      0.81      0.80      2400
weighted avg       0.82      0.82      0.82      2400


🧠 Random Forest
Accuracy: 0.80125
              precision    recall  f1-score   support

           0       0.81      0.56      0.66       825
           1       0.80      0.93      0.86      1575

    accuracy                           0.80      24

In [25]:
best_model

('Naive Bayes',
 Pipeline(steps=[('cv',
                  CountVectorizer(max_features=3000, ngram_range=(1, 2),
                                  stop_words='english')),
                 ('clf', MultinomialNB())]))

In [26]:
best_model[1].fit(X_train, y_train)
y_pred = best_model[1].predict(X_test)
y_pred

array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [27]:
test_review = "A beautifully written story with rich characters and an inspiring message — I couldn’t put it down!"
test_review = re.sub(r"[^a-zA-Z0-9\s]", "", test_review).lower()
test_review = " ".join([word for word in test_review.split() if word not in stopwords.words('english')])
test_review=re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(test_review))
test_review=BeautifulSoup(test_review, 'lxml').get_text()
text_review= " ".join(test_review.split())
test_review=[" ".join([lemmatizer.lemmatize(word) for word in test_review.split()])]
print(test_review)
rating = best_model[1].predict(test_review)[0]

print(rating)
print("positive" if rating == 1 else "negative")

['beautifully written story rich character inspiring message couldnt put']
1
positive


In [28]:
test_review = "This was the worst book I’ve ever read—boring, repetitive, and a complete waste of time."
test_review = re.sub(r"[^a-zA-Z0-9\s]", "", test_review).lower()
test_review = " ".join([word for word in test_review.split() if word not in stopwords.words('english')])
test_review=re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(test_review))
test_review=BeautifulSoup(test_review, 'lxml').get_text()
test_review=[" ".join([lemmatizer.lemmatize(word) for word in test_review.split()])]
print(test_review)
rating = best_model[1].predict(test_review)[0]

print(rating)
print("positive" if rating == 1 else "negative")

['worst book ive ever readboring repetitive complete waste time']
0
negative


## method 2

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [31]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [34]:
from sklearn.naive_bayes import MultinomialNB
model_bow=MultinomialNB().fit(X_train_bow,y_train)
model_tfidf=MultinomialNB().fit(X_train_tfidf,y_train)

In [35]:
y_pred_bow=model_bow.predict(X_test_bow)
y_pred_tfidf=model_bow.predict(X_test_tfidf)

In [36]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [37]:
confusion_matrix(y_test,y_pred_bow)

array([[ 544,  281],
       [ 121, 1454]], dtype=int64)

In [38]:
confusion_matrix(y_test,y_pred_tfidf)

array([[ 383,  442],
       [  55, 1520]], dtype=int64)

In [39]:
print(classification_report(y_test, y_pred_bow))

              precision    recall  f1-score   support

           0       0.82      0.66      0.73       825
           1       0.84      0.92      0.88      1575

    accuracy                           0.83      2400
   macro avg       0.83      0.79      0.80      2400
weighted avg       0.83      0.83      0.83      2400



In [40]:
print(classification_report(y_test, y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.87      0.46      0.61       825
           1       0.77      0.97      0.86      1575

    accuracy                           0.79      2400
   macro avg       0.82      0.71      0.73      2400
weighted avg       0.81      0.79      0.77      2400



In [41]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

BOW accuracy:  0.8325
TFIDF accuracy:  0.7929166666666667


In [44]:
test_review = "A beautifully written story with rich characters and an inspiring message — I couldn’t put it down!"
test_review = re.sub(r"[^a-zA-Z0-9\s]", "", test_review).lower()
test_review = " ".join([word for word in test_review.split() if word not in stopwords.words('english')])
test_review=re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(test_review))
test_review=BeautifulSoup(test_review, 'lxml').get_text()
text_review= " ".join(test_review.split())
test_review=[" ".join([lemmatizer.lemmatize(word) for word in test_review.split()])]
test_review=bow.transform(test_review).toarray()
rating = model_bow.predict(test_review)[0]

print(rating)
print("positive" if rating == 1 else "negative")

1
positive


In [45]:
test_review = "This was the worst book I’ve ever read—boring, repetitive, and a complete waste of time."
test_review = re.sub(r"[^a-zA-Z0-9\s]", "", test_review).lower()
test_review = " ".join([word for word in test_review.split() if word not in stopwords.words('english')])
test_review=re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(test_review))
test_review=BeautifulSoup(test_review, 'lxml').get_text()
text_review= " ".join(test_review.split())
test_review=[" ".join([lemmatizer.lemmatize(word) for word in test_review.split()])]
test_review=bow.transform(test_review).toarray()
rating = model_bow.predict(test_review)[0]

print(rating)
print("positive" if rating == 1 else "negative")

0
negative
